## Joins with `sqlalchemy`

To join to tables in `sqlalchemy`

1. Use `join` to create a `Join` object
2. Build a `select` statement from the `join` object

## Example - Reading in the Company `db`

In [1]:
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, func
from sqlalchemy.ext.automap import automap_base
import pandas as pd
from dfply import *
from sqlalchemy import select as selectq
from sqlalchemy import join
from sqlalchemy import func, select as selectq
from toolz import first, get
from sqlalchemy import Integer, Float, String, DateTime
from more_sqlalchemy import pprint

In [2]:
engine = create_engine("sqlite:///databases/company_2_7_2.db")

Base = automap_base()
Base.prepare(engine, reflect=True)
Dept = Base.classes.department
Empl = Base.classes.employee

AttributeError: department

In [ ]:

d = selectq([Dept])
pd.read_sql_query(d, con=engine)

In [ ]:
e = selectq([Empl])
pd.read_sql_query(e, con=engine)

## Using `sqlalchemy.join` to create a `Join`

**Syntax:** `join(left_table, right_table, onclause=left_table.column == right_table.column)`

* Defaults to an inner join
* Set `isouter=True` to get a `LEFT OUTER JOIN`
* Set `full=True` to get a `FULL OUTER JOIN`

In [ ]:
j = join(Empl, Dept, onclause=Empl.DeptID == Dept.DeptID)
print(j)

## Inspecting the joined column names

Note that the columns are renamed `tableName_columnName`.

In [3]:
j.c.keys()

NameError: name 'j' is not defined

## Creating a `Select` expression for a `Join`

In [9]:
stmt = selectq([j])
print(stmt)

SELECT employee."DeptID", employee."LastName", employee."EmpID", department."DeptID", department."DeptName" 
FROM employee JOIN department ON employee."DeptID" = department."DeptID"


In [10]:
pd.read_sql_query(stmt, con=engine)

,DeptID,LastName,EmpID,DeptID,DeptName
0,31.0,Rafferty,1,31,Sales
1,33.0,Jones,2,33,Engineering
2,33.0,Heisenberg,3,33,Engineering
3,34.0,Robinson,4,34,Clerical
4,34.0,Smith,5,34,Clerical


## Left Join

In [11]:
left_join = join(Empl, Dept, onclause=Empl.DeptID==Dept.DeptID, isouter=True)
left_join_stmt = selectq([left_join])
pd.read_sql_query(left_join_stmt, con=engine)

,DeptID,LastName,EmpID,DeptID,DeptName
0,31.0,Rafferty,1,31.0,Sales
1,33.0,Jones,2,33.0,Engineering
2,33.0,Heisenberg,3,33.0,Engineering
3,34.0,Robinson,4,34.0,Clerical
4,34.0,Smith,5,34.0,Clerical
5,NaN,Williams,6,NaN,None


## Right Join

To get a `RIGHT OUTER JOIN`, just switch the order and use a `LEFT OUTER JOIN`

In [12]:
right_join = join(Dept, Empl, onclause=Empl.DeptID==Dept.DeptID, isouter=True)
right_join_stmt = selectq([right_join])
pd.read_sql_query(right_join_stmt, con=engine)

,DeptID,DeptName,DeptID,LastName,EmpID
0,31,Sales,31.0,Rafferty,1.0
1,33,Engineering,33.0,Heisenberg,3.0
2,33,Engineering,33.0,Jones,2.0
3,34,Clerical,34.0,Robinson,4.0
4,34,Clerical,34.0,Smith,5.0
5,35,Marketing,NaN,None,NaN


## Full Outer Join

**Note:** `sqllite` does not support this type of join `:/`

In [13]:
full_join = join(Empl, Dept, onclause=Empl.DeptID==Dept.DeptID, full=True)
full_join_stmt = selectq([full_join])
pd.read_sql_query(full_join_stmt, con=engine)

OperationalError: (sqlite3.OperationalError) RIGHT and FULL OUTER JOINs are not currently supported [SQL: 'SELECT employee."DeptID", employee."LastName", employee."EmpID", department."DeptID", department."DeptName" \nFROM employee FULL OUTER JOIN department ON employee."DeptID" = department."DeptID"'] (Background on this error at: http://sqlalche.me/e/e3q8)

## <font color="red"> Exercise 3 </font>

Determine all the players that have hit more than 50 home runs in a season.  The final table should include the players proper name, as well as the team name.  

**Hint:** You will need join the files listed below.  To get credit for this exercise, you will need to create a database containing these three tables and use the `sqlalchemy` join methods presented above.

In [4]:
DTYPES_TO_SQLALCHEMY_TYPES = {'O':String,
                              'i':Integer,
                              'f':Float,
                              'M':DateTime}

def get_sql_types(df):
    get_sqltype = lambda dtype : get(dtype.kind,DTYPES_TO_SQLALCHEMY_TYPES)
    zip_col_type = lambda df : zip(df.columns,df.dtypes)
    return {col:get_sqltype(dtype) for col,dtype in zip_col_type(df)}

In [5]:
files = ("./data/baseball/core/Batting.csv", 
        "./data/baseball/core/People.csv",
        "./data/baseball/core/Teams.csv")

In [6]:
batting, people, teams = [pd.read_csv(f) for f in files]

In [7]:
batting_select = (batting
                >> select(X.playerID,X.yearID,X.teamID,X.HR)
                >>mutate(id = X.index))
people_select = (people
                >> select(X.playerID,X.nameFirst,X.nameLast)
                >>mutate(id = X.index))
team_select = (teams
              >> select(X.teamID, X.name,X.yearID)
              >>mutate(id = X.index))

In [42]:
!rm ./databases/baseball_2_7_4.db

rm: cannot remove './databases/baseball_2_7_4.db': No such file or directory


In [45]:
engine_baseball = create_engine("sqlite:///databases/baseball_2_7_4.db")

In [44]:
sql_types_batting = get_sql_types(batting_select)
sql_types_people = get_sql_types(people_select)
sql_types_team = get_sql_types(team_select)

In [46]:
schema = pd.io.sql.get_schema(batting_select, # dataframe
                              'batting', # name in SQL db
                              keys='id', # primary key
                              con=engine, # connection
                              dtype=sql_types_batting # SQL types
                             )
print(schema)


CREATE TABLE batting (
	"playerID" VARCHAR, 
	"yearID" INTEGER, 
	"teamID" VARCHAR, 
	"HR" INTEGER, 
	id INTEGER NOT NULL, 
	CONSTRAINT batting_pk PRIMARY KEY (id)
)




In [47]:
engine_baseball.execute(schema)

In [48]:
schema = pd.io.sql.get_schema(people_select, # dataframe
                              'people', # name in SQL db
                              keys='id', # primary key
                              con=engine, # connection
                              dtype=sql_types_people # SQL types
                             )
print(schema)


CREATE TABLE people (
	"playerID" VARCHAR, 
	"nameFirst" VARCHAR, 
	"nameLast" VARCHAR, 
	id INTEGER NOT NULL, 
	CONSTRAINT people_pk PRIMARY KEY (id)
)




In [49]:
engine_baseball.execute(schema)

In [50]:
schema = pd.io.sql.get_schema(team_select, # dataframe
                              'team', # name in SQL db
                              keys='id', # primary key
                              con=engine, # connection
                              dtype=sql_types_team # SQL types
                             )
print(schema)


CREATE TABLE team (
	"teamID" VARCHAR, 
	name VARCHAR, 
	"yearID" INTEGER, 
	id INTEGER NOT NULL, 
	CONSTRAINT team_pk PRIMARY KEY (id)
)




In [51]:
engine_baseball.execute(schema)

In [52]:
batting_select.to_sql('batting', 
                  con=engine_baseball, 
                  dtype=sql_types_batting, 
                  index=False,
                  if_exists='append')

In [53]:
people_select.to_sql('people', 
                  con=engine_baseball, 
                  dtype=sql_types_people, 
                  index=False,
                  if_exists='append')

In [54]:
team_select.to_sql('team', 
                  con=engine_baseball, 
                  dtype=sql_types_team, 
                  index=False,
                  if_exists='append')

In [55]:

Base = automap_base()
Base.prepare(engine_baseball, reflect=True)
Batting = Base.classes.batting
People = Base.classes.people
Teams = Base.classes.team

In [56]:
from sqlalchemy import inspect
Session = sessionmaker(bind=engine2)
session = Session()

insp = inspect(engine2)
insp.get_table_names()

['batting', 'people', 'team']

In [57]:
b = selectq([Batting])
p = selectq([People])
t = selectq([Teams])

In [119]:
batting_group = (selectq([Batting.yearID,
                         Batting.teamID,
                         Batting.playerID,
                         func.sum(Batting.HR).label('yearHRs')
                        ])
                .group_by('playerID', 'yearID', 'teamID')
                )

In [136]:
batting_group.c.keys()

['yearID', 'teamID', 'playerID', 'yearHRs']

In [199]:
batting_filter = (selectq([batting_group])
                 .where(batting_group.c.yearHRs>=50).alias('b'))

In [200]:
batting_filter.c.keys()

['yearID', 'teamID', 'playerID', 'yearHRs']

In [201]:
people_select = (selectq([People.playerID,People.nameFirst,People.nameLast]).distinct().alias('p'))

In [238]:
people_select.c.keys()

['playerID', 'nameFirst', 'nameLast']

In [255]:
team_select = (selectq([Teams.teamID,Teams.name,Teams.yearID]).distinct().alias('t'))

In [256]:
team_select.c.keys()

['teamID', 'name', 'yearID']

In [205]:
bat_peop_join = join(batting_filter,people_select, onclause=batting_filter.c.playerID==people_select.c.playerID, isouter=False)
bat_peop_join.c.keys()

['b_yearID',
 'b_teamID',
 'b_playerID',
 'b_yearHRs',
 'p_playerID',
 'p_nameFirst',
 'p_nameLast']

In [247]:
batting_filter_join_people = (selectq([bat_peop_join.c.b_yearID,bat_peop_join.c.b_teamID,bat_peop_join.c.b_yearHRs,bat_peop_join.c.p_nameFirst,bat_peop_join.c.p_nameLast])
                              .select_from(bat_peop_join).alias('bp'))

In [248]:
batting_filter_join_people.c.keys()

['yearID', 'teamID', 'yearHRs', 'nameFirst', 'nameLast']

In [257]:
bp_t_join = join(batting_filter_join_people, team_select, 
                 onclause=(batting_filter_join_people.c.teamID==team_select.c.teamID) & (batting_filter_join_people.c.yearID==team_select.c.yearID), isouter=False)

In [258]:
bp_t_join.c.keys()

['bp_yearID',
 'bp_teamID',
 'bp_yearHRs',
 'bp_nameFirst',
 'bp_nameLast',
 't_teamID',
 't_name',
 't_yearID']

In [259]:
bpt_joined = (selectq([bp_t_join.c.bp_yearID,bp_t_join.c.bp_yearHRs,bp_t_join.c.bp_nameFirst,bp_t_join.c.bp_nameLast,bp_t_join.c.t_name]).select_from(bp_t_join))

In [262]:
#pprint(bpt_joined)

In [261]:
pd.read_sql_query(bpt_joined, con=engine_baseball)

,yearID,yearHRs,nameFirst,nameLast,name
0,1996,50,Brady,Anderson,Baltimore Orioles
1,2010,54,Jose,Bautista,Toronto Blue Jays
2,1995,50,Albert,Belle,Cleveland Indians
3,2001,73,Barry,Bonds,San Francisco Giants
4,2013,53,Chris,Davis,Baltimore Orioles
5,1990,51,Cecil,Fielder,Detroit Tigers
6,2007,50,Prince,Fielder,Milwaukee Brewers
7,1977,52,George,Foster,Cincinnati Reds
8,1932,58,Jimmie,Foxx,Philadelphia Athletics
9,1938,50,Jimmie,Foxx,Boston Red Sox


## Up Next

Stuff